In [1]:
import numpy as np
import pandas as pd
import sqlite3

# import polarity data

In [32]:
user_polarity = pd.read_csv("USER_POLARITY_BARBERA.txt", delimiter = "\t", header=None)
user_polarity = user_polarity.drop([2], axis=1)
user_polarity = user_polarity.rename(columns={0: "user_ID", 1: "polarity_score"})
user_polarity.head(5)

,user_ID,polarity_score
0,1000010898,-1.375595
1,1000015790,1.817662
2,1000044026,1.817662
3,100004471,-0.798651
4,100004662,-1.618781


In [33]:
user_production_score = pd.read_csv("obamacare/users_production_scores.txt", 
                                    delimiter = "\t", header = None, lineterminator='\n')
user_production_score = user_production_score.rename(columns={0: "user_ID", 1: "production_score"})
user_production_score = user_production_score.drop([2,3], axis=1)
user_production_score.head(5)

,user_ID,production_score
0,334537201,0.734128
1,115094250,0.226682
2,2369916919,0.288123
3,1146470090,0.305678
4,95708744,0.347375


In [34]:
user_consumption_score = pd.read_csv("obamacare/users_consumption_scores.txt", 
                                     delimiter = "\t", header = None, lineterminator='\n')
user_consumption_score = user_consumption_score.rename(columns={0: "user_ID", 1: "consumption_score"})
user_consumption_score = user_consumption_score.drop([2,3], axis=1)
user_consumption_score.head(5)

,user_ID,consumption_score
0,334537201,0.738397
1,115094250,0.391875
2,2369916919,0.392721
3,1146470090,0.381466
4,95708744,0.347500


# test blocks

In [2]:
# test = pd.read_csv("obamacare/USER_TWEETS.txt.gz", compression = "gzip",
#                    delimiter = "\t",
#                    header = None, 
#                    lineterminator = "\n",
#                    error_bad_lines=False, nrows=1000000)

In [ ]:
# test

# data cleaning

In [59]:
def prepare_df(df):
    df = df.drop(columns=[1, 4, 10, 11, 12, 13], axis=1)
    
    # the user has polarity score, production score, consumption score
    # and the tweet cites some domain
    df = df[df[0].isin(user_polarity["user_ID"]) & \
            df[0].isin(user_production_score["user_ID"]) & \
            df[0].isin(user_consumption_score["user_ID"]) & \
            df[9].notna()].dropna()
    
    df = df.rename(columns={
        0:  "user_ID",
#         1:  "tweet_ID",
        2:  "tweet_content",
        3:  "date",
#         4:  "platform",
        5:  "likes",        # my speculation
        6:  "comments",     # my speculation
        7:  "retweets",     # my speculation
        8:  "hashtag",
        9:  "domain",
#         10: "ppl_mentioned"
        })
    return df

# create database

In [68]:
conn = sqlite3.connect("obama.db") 
df_iter = pd.read_csv("obamacare/USER_TWEETS.txt.gz", 
                      delimiter = "\t", 
                      compression = "gzip",
                      header = None, 
                      lineterminator='\n',
                      error_bad_lines=False, 
                      chunksize = 100000,
                      names=list(range(14)))
for df in df_iter:
    df = prepare_df(df)
    if df is not None:
        df.to_sql("tweets", conn, if_exists = "append", index = False)

/Users/wangyuqi/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/wangyuqi/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (5,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/wangyuqi/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (5,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/wangyuqi/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (5,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_rai

In [71]:
cursor = conn.cursor()

In [73]:
q = cursor.execute("SELECT COUNT(*) FROM tweets")
result = cursor.fetchone()
result

(1915866,)

In [ ]:
# df = pd.read_sql_query(cmd, conn)